In [14]:
import pandas as pd
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import chi2
import matplotlib.dates as mdates
import random
from sklearn.model_selection import train_test_split

In [15]:
len(['adfadf'])

1

In [16]:
df_l8 = np.array([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
df_l8==1

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False, False, False, False],
       [ True, False, False,  True,  True, False, False],
       [ True, False, False, False, False,  True,  True],
       [False,  True, False,  True, False,  True, False],
       [False,  True, False, False,  True, False,  True],
       [False, False,  True,  True, False, False,  True],
       [False, False,  True, False,  True,  True, False]])

In [17]:
np.zeros(12)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
df_l8 = pd.DataFrame([
        [1,1,1,1,1,1,1],
        [1,1,1,2,2,2,2],
        [1,2,2,1,1,2,2],
        [1,2,2,2,2,1,1],
        [2,1,2,1,2,1,2],
        [2,1,2,2,1,2,1],
        [2,2,1,1,2,2,1],
        [2,2,1,2,1,1,2]
        ])
(df_l8==1).values

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False, False, False, False],
       [ True, False, False,  True,  True, False, False],
       [ True, False, False, False, False,  True,  True],
       [False,  True, False,  True, False,  True, False],
       [False,  True, False, False,  True, False,  True],
       [False, False,  True,  True, False, False,  True],
       [False, False,  True, False,  True,  True, False]])

## データの取得

In [19]:
df = pd.read_csv('../data/letter_recognition.csv', header=None)

#Aのみを判定するため，Aを１，A以外を0にした．
df[0] = df[0].apply(lambda x: 1 if x == 'A' else 0)

#Xとyを入力
X = df[range(1,17)]
y = df[0]



In [20]:
#バギング側の話
#ブートストラップサンプリングの個数
n = 10
seed = random.randint(0, n)

#使用する7つの変数をランダムに取得する
random.seed(1)
random_s = random.sample(list(X.columns), 7)
use_X = X[random_s]

X_train, X_test, y_train, y_test = train_test_split(use_X, y, test_size=0.2)


### Xとyが入力されたら自動的にMDを求めるためのscalerとinv_covと使用する変数(select_columns)を出力するアルゴリズムを作る

In [21]:
normal_idx = y_train[y_train == 0].index.to_list()
anomaly_idx = y_train[y_train == 1].index.to_list()

#通常状態の説明変数と目的変数を定義
normal_X = X_train.loc[normal_idx]
normal_y = y_train.loc[normal_idx]
anomaly_X = X_train.loc[anomaly_idx]
anomaly_y = y_train.loc[anomaly_idx]

In [22]:
normal_X.to_csv('../data/normal_X.csv')

## 必要な関数の定義

In [23]:
#各説明変数ごとに標準化
def transform_standard(fit_X, transform_X):
    scaler = StandardScaler()
    scaler.fit(fit_X)
    return scaler.transform(transform_X)

#共分散行列の逆行列
def inv_cov(x):
    #標準化後のベクトルを入力する
    #標準化した後なので相関行列と分散共分散行列は一致する
    cov = np.cov(x.T)
    return np.linalg.inv(cov)

#マハラノビス汎距離
def cal_MD(Z, inv_C, k):
    MD = []
    for i in range(len(Z)):
        _a = np.dot(Z[i], inv_C)
        _MD = np.dot(_a, Z[i].T)
        _MD = _MD / k
        MD.append(_MD)
    return MD


In [24]:
#標準化
normal_Z = transform_standard(normal_X, normal_X)
anomaly_Z = transform_standard(normal_X, anomaly_X) #異常データも正常データの平均と分散で標準化

#共分散行列の逆行列
inv_C = inv_cov(normal_Z)

#マハラノビス距離の計算
normal_MD = cal_MD(normal_Z, inv_C, len(use_X.columns))
anomaly_MD = cal_MD(anomaly_Z, inv_C, len(use_X.columns))


## ここまではあってる

### 正常データの中の異常なデータを任意に削除することもある

In [25]:
# def del_anomaly(x):
#     #マハラノビス距離がx以上の通常データを削除する
#     df_MD = pd.DataFrame(index=normal_idx, data=normal_MD)
#     del_anomaly_idx = df_MD.loc[df_MD[0] > x].index.to_list()
#     for i in del_anomaly_idx:
#         normal_idx.remove(i)
#     return normal_idx
# len(del_anomaly(3))

In [26]:
#L8直行表
df_l8 = pd.DataFrame([[1,1,1,1,1,1,1],[1,1,1,2,2,2,2],[1,2,2,1,1,2,2],[1,2,2,2,2,1,1],[2,1,2,1,2,1,2],[2,1,2,2,1,2,1],[2,2,1,1,2,2,1],[2,2,1,2,1,1,2]])
l8 = (df_l8==1).values
l8

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False, False, False, False],
       [ True, False, False,  True,  True, False, False],
       [ True, False, False, False, False,  True,  True],
       [False,  True, False,  True, False,  True, False],
       [False,  True, False, False,  True, False,  True],
       [False, False,  True,  True, False, False,  True],
       [False, False,  True, False,  True,  True, False]])

In [27]:
#異常データのマハラノビス距離
result = np.zeros((l8.shape[0], anomaly_Z.shape[0]))
for x, l8_row in enumerate(l8):
    result[x] = cal_MD(anomaly_Z[:, l8_row], inv_C[l8_row][:,l8_row], sum(l8_row))
result.shape

(8, 632)

In [28]:
result

array([[2.75149986, 0.6863897 , 1.11114997, ..., 2.23161942, 0.50717266,
        0.76723311],
       [5.38600733, 1.91953007, 2.02219593, ..., 3.51199853, 1.05568758,
        1.44005027],
       [0.89127751, 0.63813963, 0.19037882, ..., 0.95093923, 0.23735601,
        0.84675589],
       ...,
       [1.26904017, 0.99559578, 0.67356007, ..., 2.36464374, 0.30629097,
        0.84883279],
       [4.95846682, 1.38296956, 1.03195536, ..., 4.24281101, 0.43246595,
        1.05876001],
       [4.88249967, 1.65734469, 1.77010215, ..., 4.00819682, 0.46226945,
        1.64171622]])

In [29]:
#SN比
sn = np.zeros(l8.shape[0])
for idx, row in enumerate(result):
    sum_MD = 0
    for i in range(len(row)):
        sum_MD += 1 / row[i]
    sn[idx] = -10 * math.log10(sum_MD / len(row))
sn

array([-1.74895568,  0.42101705, -5.22365946, -4.9906142 , -6.97455843,
       -6.17204726, -2.50264905,  0.4189587 ])

In [30]:
#変数選択
df_sn = pd.DataFrame(index=X_train.columns, columns=['SN比','残す'])
for i, clm in enumerate(X_train.columns):
    df_sn.loc[df_sn.index == clm, 'SN比'] = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]])
    df_sn.loc[df_sn.index == clm, '残す'] = sum(sn[l8.T[i]]) - sum(sn[~l8.T[i]]) > 0
df_sn
# select_v = df_sn.reset_index()

,SN比,残す
5,3.688084,True
10,-2.17658,False
14,19.94925,True
13,-6.127137,False
2,1.321101,True
16,0.182169,True
12,-4.056024,False


In [31]:
normal_X_d = normal_X.drop(columns=df_sn[df_sn['残す'] == False].index)
anomaly_X_d = anomaly_X.drop(columns=df_sn[df_sn['残す'] == False].index)

normal_Z_d = transform_standard(normal_X_d, normal_X_d)
anomaly_Z_d = transform_standard(normal_X_d, anomaly_X_d)

#共分散行列の逆行列
inv_C_d = inv_cov(normal_Z_d)

#マハラノビス距離の計算
normal_MD = cal_MD(normal_Z_d, inv_C_d, len(normal_X_d.columns))
anomaly_MD = cal_MD(anomaly_Z_d, inv_C_d, len(normal_X_d.columns))

In [40]:
type(df_sn['SN比'].astype(float).idxmax()) == np.int64

True

In [32]:
Z = transform_standard(normal_X_d, X_test.drop(columns=df_sn[df_sn['残す'] == False].index))
inv_C_d = inv_cov(normal_Z_d)

MD = cal_MD(Z, inv_C_d, len(normal_X_d.columns))
MD

[1.1297457169311214,
 1.3365514092529032,
 0.9607023253953421,
 1.3494947322586315,
 2.3542549832726443,
 0.4867335902866202,
 0.4903156109123312,
 1.0515220534644287,
 0.5326373957585221,
 0.11495887308451114,
 0.8747874464373945,
 1.077863691385431,
 0.46635937717818016,
 0.10042376641451503,
 0.4903156109123312,
 1.1552578632031447,
 0.7351531535388649,
 1.4268758675118547,
 0.10660404071499902,
 0.41009384126559056,
 0.3558793344102121,
 0.2046160374128236,
 1.3458917599548774,
 1.0292295658235078,
 0.8344904114711027,
 0.5225432786002568,
 0.1566443204530258,
 3.7830772260487926,
 0.7819662690514304,
 0.47332638559726087,
 1.9155833426891073,
 0.7271123116531393,
 1.222001469066923,
 0.28672499894564885,
 0.6754125667039245,
 1.6644621233269947,
 0.7603329157037718,
 0.4924718254741995,
 1.1552578632031447,
 1.1518194471290772,
 1.9783974865967833,
 0.12342115472075676,
 2.748634883420238,
 1.1276145128335981,
 0.5865525119349492,
 1.2058702725519657,
 1.1188519443375577,
 0.95227

In [33]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test.values, MD) 

0.6488445365964421

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [35]:
clf = LogisticRegression(solver="liblinear", random_state=0).fit(use_X, y)
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.9099612000311593

In [36]:
confusion_matrix(y_test, clf.predict(X_test))

array([[3830,   13],
       [ 149,    8]], dtype=int64)